# [중요] 리소스 정리 (Clean-up Resource)
### 아래 과정을 꼼꼼하게 해주세요. 그렇지 않으면 요금이 발생할 수 있습니다.

# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./2_Inference/src')

# 2. 세이지 메이커 앤드포인트 삭제 확인
- 아래와 같이 콘솔에 가셔서 엔드포인트가 없어야 합니다.
    - 만일 존재하면 삭제 해주세요.

In [ ]:
from inference_utils import delete_endpoint
import boto3

In [ ]:
client = boto3.Session().client('sagemaker')

endpoint_name = 'sm-ncf'

response = client.list_endpoints(
    SortBy='Name',
    SortOrder='Ascending',
    NameContains=endpoint_name,
)

for ep_list in response['Endpoints']:
    print("endpoint name: \n", ep_list["EndpointName"])
    ep = ep_list["EndpointName"]
    delete_endpoint(client, ep)

In [ ]:
endpoint_name = 'lambda-deploy-endpoint'

response = client.list_endpoints(
    SortBy='Name',
    SortOrder='Ascending',
    NameContains=endpoint_name,
)

for ep_list in response['Endpoints']:
    print("endpoint name: \n", ep_list["EndpointName"])
    ep = ep_list["EndpointName"]
    delete_endpoint(client, ep)


- SageMaker Concole --> 왼쪽 메뉴의 Inference --> Endpoints 로 다음과 같이 엔드포인트가 있는지 확인 하세요.

![sm_endpoint_console.png](img/sm_endpoint_console.png)

# 3. 파이프라인 삭제

In [ ]:
sagemaker_boto_client = boto3.client('sagemaker')

def clean_pipeline(pipeline_name, isDeletePipeline=False, verbose=False):
    '''
    파이프라인 삭제
    pipeline_name = 'sagemaker-pipeline-step-by-step-phase01'
    clean_pipeline(pipeline_name = pipeline_name, isDeletePipeline=False, verbose=False)   
    '''
    # project_prefix 의 prefix 를 가진 파이프라인을 모두 가져오기
    response = sagemaker_boto_client.list_pipelines(
        PipelineNamePrefix= pipeline_name,
        SortBy= 'Name',    
        SortOrder='Descending',
        #NextToken='string',
        MaxResults=100
    )

    if verbose:
        print(f"\n### Display pipelines with this prefix {pipeline_name} \n")        
        dp(response)

    
    # pipeline_name 보여주기
    if any(pipeline["PipelineDisplayName"] == pipeline_name for pipeline in response["PipelineSummaries"]):
        print(f"pipeline {pipeline_name} exists")
        response = sagemaker_boto_client.describe_pipeline(
            PipelineName= pipeline_name
        )    
    
        if verbose:
            print(f"\n### pipeline {pipeline_name} definiton is \n")
            dp(response)
            
        if isDeletePipeline:
            sagemaker_boto_client.delete_pipeline(PipelineName= pipeline_name)            
            print(f"pipeline {pipeline_name} is deleted")            

    else:
        print(f"pipeline {pipeline_name} doesn't exists")

## 3.1. 훈련 파이프라인 삭제

In [ ]:


sm_training_pipeline = 'ncf-pipeline-nb-training'

clean_pipeline(pipeline_name = sm_training_pipeline, isDeletePipeline=True, verbose=False)   


## 3.2 추론 파이프라인 삭제

In [ ]:

sm_serving_pipeline = 'ncf-sm-serving-pipeline'

clean_pipeline(pipeline_name = sm_serving_pipeline, isDeletePipeline=True, verbose=False)   


# 4. 모델 패키지 삭제

- 해당 패키지 그룹을 삭제 함.
- 추가적으로 다른 채키지 그룹은 리스트로 보여 줌.

In [ ]:
sm_client = boto3.Session().client('sagemaker')
model_package_group_name = 'NCFModel'

def delete_model_package_group(sm_client, ModelPackageGroupName, isDelete=False, verbose=True):
    
    for mpg in sm_client.list_model_package_groups()['ModelPackageGroupSummaryList']:
        print(mpg['ModelPackageGroupName'])

        if (mpg['ModelPackageGroupName'] == ModelPackageGroupName): # 해당 패키지만 그룹만 삭제
            for mp in sm_client.list_model_packages(ModelPackageGroupName=mpg['ModelPackageGroupName'])['ModelPackageSummaryList']:
                print(mp['ModelPackageArn'])
                try:
                    if isDelete:
                        sm_client.delete_model_package(ModelPackageName=mp['ModelPackageArn'])
                        print(f"###### ModelPackageGroupName: {ModelPackageGroupName} is deleted ######")
                except BaseException as error:
                    error_msg = f"Model Package error occurred: {error}"
                    print(error_msg)  
                    pass

            try:
                pass
                if isDelete:
                    sm_client.delete_model_package_group(ModelPackageGroupName=mpg['ModelPackageGroupName'])
            except:
                print("model_pakcage_group error")
                pass
        
#delete_model_package_group(sm_client, ModelPackageGroupName=model_package_group_name, isDelete=False, verbose=True)        
delete_model_package_group(sm_client, ModelPackageGroupName=model_package_group_name, isDelete=True, verbose=True) 

# 5. 세이지 메이커 실험 지우기

In [ ]:
import time

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent

In [ ]:
experiment_name='NCFModel-single-train'  #<== 생성한 experiment_name을 넣어야 합니다.

In [ ]:
def cleanup_sme_sdk(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(
                trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                # comment out to keep trial components
                tc.delete()
            except:
                # tc is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        trial.delete()
        experiment_name = experiment.experiment_name
    experiment.delete()
    print(f"\nExperiment {experiment_name} deleted")

In [ ]:
experiment_to_cleanup = Experiment.load(
    # Use experiment name not display name
    experiment_name=experiment_name)

cleanup_sme_sdk(experiment_to_cleanup)

# 6. S3 버킷 삭제


In [ ]:
import sagemaker
sagemaker_session = sagemaker.session.Session()

In [ ]:
bucket = sagemaker_session.default_bucket()
bucket

In [ ]:
!aws s3 ls s3://$bucket/NCFModel --recursive

In [ ]:
# !aws s3 rm s3://$bucket/NCFModel --recursive

In [ ]:
!aws s3 ls s3://$bucket/NCFModel --recursive

# 7. 세이지 메이커 노트북 인스턴스 삭제
## 처음 실습시 생성한 클라우드 포메이션으로 생성한 스택을 삭제 하셨다면, 이 과정은 안하셔도 됩니다. 

## [중요]  하지만 아래의 세이지 메이커의 콘솔에 가셔서 직접 확인을 해주세요. 

## 아래와 같이 노트북 인스턴스를 선택하고, Stop (중지) 후에 Delete(삭제) 해주세요.

![notebook_instance.png](img/notebook_instance.png)

## 아래와 같이 위의 노트북 인스턴스가 없어야 합니다.

![empty_notebook.png](img/empty_notebook.png)